# Imports

In [1]:
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, \
                         TrainingArguments, Trainer
from torch.optim import AdamW

/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load and Process Dataset

In [2]:
raw_datasets = load_dataset("wnut_17")

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

In [4]:
raw_datasets["train"][0]["tokens"]

['@paulwalk',
 'It',
 "'s",
 'the',
 'view',
 'from',
 'where',
 'I',
 "'m",
 'living',
 'for',
 'two',
 'weeks',
 '.',
 'Empire',
 'State',
 'Building',
 '=',
 'ESB',
 '.',
 'Pretty',
 'bad',
 'storm',
 'here',
 'last',
 'evening',
 '.']

In [5]:
raw_datasets["train"][0]["ner_tags"]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 8,
 8,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [6]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-corporation', 'I-corporation', 'B-creative-work', 'I-creative-work', 'B-group', 'I-group', 'B-location', 'I-location', 'B-person', 'I-person', 'B-product', 'I-product'], id=None), length=-1, id=None)

In [7]:
label_names = ner_feature.feature.names
label_names

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

In [8]:
ner_feature.feature.str2int

<bound method ClassLabel.str2int of ClassLabel(names=['O', 'B-corporation', 'I-corporation', 'B-creative-work', 'I-creative-work', 'B-group', 'I-group', 'B-location', 'I-location', 'B-person', 'I-person', 'B-product', 'I-product'], id=None)>

In [9]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

@paulwalk It 's the view from where I 'm living for two weeks . Empire     State      Building   = ESB        . Pretty bad storm here last evening . 
O         O  O  O   O    O    O     O O  O      O   O   O     O B-location I-location I-location O B-location O O      O   O     O    O    O       O 


In [10]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)

In [12]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [13]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [14]:
raw_datasets["train"][0]["ner_tags"]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 8,
 8,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [15]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [16]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

In [17]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Setting Evaluation

In [18]:
metric = evaluate.load("seqeval")

In [19]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels, )
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [20]:
def evaluate_token_classification(predictions, labels, entity_types):
    predictions = np.argmax(predictions, axis=-1)
    labels = [[label_names[l] for l in label if l != -100] for label in labels]
    predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    precision_dict = {}
    recall_dict = {}
    f1_dict = {}
    support_dict = {}
    
    for entity_type in entity_types:
        precision_dict[entity_type] = {}
        recall_dict[entity_type] = {}
        f1_dict[entity_type] = {}
        support_dict[entity_type] = {}
        b_true_labels_binary = [[l if f"B-{entity_type}" == l else 'O' for l in label] for label in labels]
        b_pred_labels_binary = [[l if f"B-{entity_type}" == l else 'O' for l in label] for label in predictions]
        
        b_metrics = metric.compute(predictions=b_pred_labels_binary, references=b_true_labels_binary)
        
        precision_dict[entity_type]['B-label'] = b_metrics[entity_type]['precision']
        recall_dict[entity_type]['B-label'] = b_metrics[entity_type]['recall']
        f1_dict[entity_type]['B-label'] = b_metrics[entity_type]['f1']
        support_dict[entity_type]['B-label'] = b_metrics[entity_type]['number']
        
        i_true_labels_binary = [[l if f"I-{entity_type}" == l else 'O' for l in label] for label in labels]
        i_pred_labels_binary = [[l if f"I-{entity_type}" == l else 'O' for l in label] for label in predictions]
    
        i_metrics = metric.compute(predictions=i_pred_labels_binary, references=i_true_labels_binary)
        
        precision_dict[entity_type]['I-label'] = i_metrics[entity_type]['precision']
        recall_dict[entity_type]['I-label'] = i_metrics[entity_type]['recall']
        f1_dict[entity_type]['I-label'] = i_metrics[entity_type]['f1']
        support_dict[entity_type]['I-label'] = i_metrics[entity_type]['number']

    entity_metrics = metric.compute(predictions=predictions, references=labels)
    
    for entity_type in entity_types:
        precision_dict[entity_type]['entity'] = entity_metrics[entity_type]['precision']
        recall_dict[entity_type]['entity'] = entity_metrics[entity_type]['recall']
        f1_dict[entity_type]['entity'] = entity_metrics[entity_type]['f1']
        support_dict[entity_type]['entity'] = entity_metrics[entity_type]['number']
        
    f1_scores_list = [f1_dict[entity_type]["entity"] for entity_type in f1_dict] 
    support_list = [support_dict[entity_type]["entity"] for entity_type in support_dict]
    weights_support_list = [support / len(support_list) for support in support_list]
    
    final_dict = {
        "precision": precision_dict,
        "recall": recall_dict,
        "f1": f1_dict,
        "macro_f1": sum(f1_scores_list) / len(f1_scores_list),
        "micro_f1": sum([f1_score * support for f1_score, support in zip(f1_scores_list, weights_support_list)])
    }
    return final_dict

In [24]:
entity_types = ['corporation', 'creative-work', 'group', 'location', 'person', 'product']

# Fine-tune Model

In [22]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

### Baseline model

In [20]:
args_baseline = TrainingArguments("bert-finetuned-baseline")
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

trainer_baseline = Trainer(
    model=model,
    args=args_baseline,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

trainer_baseline.train()

test_pred_baseline = trainer_baseline.predict(test_dataset=tokenized_datasets["test"])
test_pred_baseline.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.116100
1000,0.044400


{'test_loss': 0.26021715998649597,
 'test_precision': 0.4791044776119403,
 'test_recall': 0.2974976830398517,
 'test_f1': 0.3670668953687822,
 'test_accuracy': 0.9334422110552764,
 'test_runtime': 103.7781,
 'test_samples_per_second': 12.401,
 'test_steps_per_second': 1.551}

### AdamW Optimization

In [21]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer1 = AdamW(model.parameters(), lr=2e-5)
args_adam1 = TrainingArguments(
    "bert-finetuned-adam1",
    evaluation_strategy="epoch",
)

trainer_adam1 = Trainer(
    model=model,
    args=args_adam1,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer1, None)
)

trainer_adam1.train()
test_pred_adam1 = trainer_adam1.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam1.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.167101,0.526126,0.349282,0.419842,0.915547
2,0.120500,0.180026,0.588138,0.427033,0.494802,0.922826
3,0.046500,0.173017,0.585227,0.492823,0.535065,0.927696


{'test_loss': 0.23202502727508545,
 'test_precision': 0.5274390243902439,
 'test_recall': 0.3206672845227062,
 'test_f1': 0.3988472622478386,
 'test_accuracy': 0.9359798994974874,
 'test_runtime': 10.1927,
 'test_samples_per_second': 126.267,
 'test_steps_per_second': 15.796}

In [19]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer2 = AdamW(model.parameters(), lr=2e-5)
args_adam2 = TrainingArguments(
    "bert-finetuned-adam2",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
)

trainer_adam6 = Trainer(
    model=model,
    args=args_adam2,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer2, None)
)

trainer_adam2.train()
test_pred_adam2 = trainer_adam2.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam2.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.214646,0.361538,0.056220,0.097308,0.888039
2,No log,0.188973,0.552301,0.315789,0.401826,0.910891
3,No log,0.179121,0.542808,0.379187,0.446479,0.916189


/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 0.21806401014328003,
 'test_precision': 0.45422535211267606,
 'test_recall': 0.2391102873030584,
 'test_f1': 0.31329690346083794,
 'test_accuracy': 0.931180904522613,
 'test_runtime': 135.9734,
 'test_samples_per_second': 9.465,
 'test_steps_per_second': 1.184}

In [22]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer3 = AdamW(model.parameters(), lr=0.005)
args_adam3 = TrainingArguments(
    "bert-finetuned-adam3",
    evaluation_strategy="epoch",
)

trainer_adam3 = Trainer(
    model=model,
    args=args_adam3,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer3, None)
)

trainer_adam3.train()
test_pred_adam3 = trainer_adam3.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam3.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.366915,0.000000,0.000000,0.000000,0.881670
2,0.224900,0.350126,0.000000,0.000000,0.000000,0.881670
3,0.183300,0.355093,0.000000,0.000000,0.000000,0.881670


[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'O', 'B-location'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-creative-work', 'I-creative-work', 'I-creative-work', 'I-creative-work', 'I-creative-work', 'O'], ['O', 'O', 'B-person', 'I-person', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'I-person', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-person', 'I-person', 'I-person', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'B-location', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'O', 'B-location'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-creative-work', 'I-creative-work', 'I-creative-work', 'I-creative-work', 'I-creative-work', 'O'], ['O', 'O', 'B-person', 'I-person', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'I-person', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-person', 'I-person', 'I-person', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'B-location', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'O', 'B-location'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-creative-work', 'I-creative-work', 'I-creative-work', 'I-creative-work', 'I-creative-work', 'O'], ['O', 'O', 'B-person', 'I-person', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'I-person', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-person', 'I-person', 'I-person', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'B-location', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'I-location', 'I-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-group', 'I-group', 'I-group', 'I-group', 'O', 'B-group', 'I-group', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 

{'test_loss': 0.2851142883300781,
 'test_precision': 0.0,
 'test_recall': 0.0,
 'test_f1': 0.0,
 'test_accuracy': 0.914321608040201,
 'test_runtime': 87.3455,
 'test_samples_per_second': 14.735,
 'test_steps_per_second': 1.843}

In [23]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer4 = AdamW(model.parameters(), lr=0.005)
args_adam4 = TrainingArguments(
    "bert-finetuned-adam4",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
)

trainer_adam4 = Trainer(
    model=model,
    args=args_adam4,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer4, None)
)

trainer_adam4.train()
test_pred_adam4 = trainer_adam3.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam4.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.351450,0.000000,0.000000,0.000000,0.881670
2,No log,0.373995,0.000000,0.000000,0.000000,0.881670
3,No log,0.350987,0.000000,0.000000,0.000000,0.881670


[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'O', 'B-location'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-creative-work', 'I-creative-work', 'I-creative-work', 'I-creative-work', 'I-creative-work', 'O'], ['O', 'O', 'B-person', 'I-person', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'I-person', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-person', 'I-person', 'I-person', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'B-location', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'O', 'B-location'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-creative-work', 'I-creative-work', 'I-creative-work', 'I-creative-work', 'I-creative-work', 'O'], ['O', 'O', 'B-person', 'I-person', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'I-person', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-person', 'I-person', 'I-person', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'B-location', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'I-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'O', 'B-location'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-creative-work', 'I-creative-work', 'I-creative-work', 'I-creative-work', 'I-creative-work', 'O'], ['O', 'O', 'B-person', 'I-person', 'O', 'B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-person', 'I-person', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-person', 'I-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-person', 'I-person', 'I-person', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'B-location', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location', 'I-location', 'I-location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-group', 'I-group', 'I-group', 'I-group', 'O', 'B-group', 'I-group', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 

{'test_loss': 0.2851142883300781,
 'test_precision': 0.0,
 'test_recall': 0.0,
 'test_f1': 0.0,
 'test_accuracy': 0.914321608040201,
 'test_runtime': 10.2674,
 'test_samples_per_second': 125.348,
 'test_steps_per_second': 15.681}

In [24]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer5 = AdamW(model.parameters())
args_adam5 = TrainingArguments(
    "bert-finetuned-adam5",
    evaluation_strategy="epoch",
)

trainer_adam5 = Trainer(
    model=model,
    args=args_adam5,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer5, None)
)

trainer_adam5.train()
test_pred_adam5 = trainer_adam5.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam5.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.356571,0.000000,0.000000,0.000000,0.881670
2,0.225600,0.345084,0.000000,0.000000,0.000000,0.881670
3,0.182200,0.343529,0.000000,0.000000,0.000000,0.881670


/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/arsen/miniconda3/envs/text_mi


KeyboardInterrupt



In [23]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer6 = AdamW(model.parameters(), lr=1e-5)
args_adam6 = TrainingArguments(
    "bert-finetuned-adam6",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
)

trainer_adam6 = Trainer(
    model=model,
    args=args_adam6,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer6, None)
)

trainer_adam6.train()
test_pred_adam6 = trainer_adam6.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam6.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.244335,0.000000,0.000000,0.000000,0.881670
2,No log,0.222067,0.581560,0.098086,0.167861,0.892267
3,No log,0.210411,0.583039,0.197368,0.294906,0.898689


/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 0.2241528481245041,
 'test_precision': 0.44649446494464945,
 'test_recall': 0.11214087117701575,
 'test_f1': 0.17925925925925928,
 'test_accuracy': 0.9230150753768844,
 'test_runtime': 139.6398,
 'test_samples_per_second': 9.217,
 'test_steps_per_second': 1.153}

In [25]:
macro_micro_adam6 = evaluate_token_classification(predictions=test_pred_adam6.predictions, \
                                                  labels=test_pred_adam6.label_ids, \
                                                  entity_types=entity_types)
macro_micro_adam6

{'precision': {'corporation': {'B-label': 0.0, 'I-label': 0.0, 'entity': 0.0},
  'creative-work': {'B-label': 0.0, 'I-label': 0.0, 'entity': 0.0},
  'group': {'B-label': 0.0, 'I-label': 0.0, 'entity': 0.0},
  'location': {'B-label': 0.0, 'I-label': 0.037037037037037035, 'entity': 0.0},
  'person': {'B-label': 0.011235955056179775,
   'I-label': 0.011976047904191617,
   'entity': 0.0},
  'product': {'B-label': 0.0, 'I-label': 0.0, 'entity': 0.0}},
 'recall': {'corporation': {'B-label': 0.0, 'I-label': 0.0, 'entity': 0.0},
  'creative-work': {'B-label': 0.0, 'I-label': 0.0, 'entity': 0.0},
  'group': {'B-label': 0.0, 'I-label': 0.0, 'entity': 0.0},
  'location': {'B-label': 0.0, 'I-label': 0.010526315789473684, 'entity': 0.0},
  'person': {'B-label': 0.002331002331002331,
   'I-label': 0.005952380952380952,
   'entity': 0.0},
  'product': {'B-label': 0.0, 'I-label': 0.0, 'entity': 0.0}},
 'f1': {'corporation': {'B-label': 0.0, 'I-label': 0.0, 'entity': 0.0},
  'creative-work': {'B-label'